# Analyse von Luftverschmutzungsdaten

## Daten
- Quelle: die [API](https://aqs.epa.gov/aqsweb/documents/data_api.html#lists) der EPA (US-Umweltbehörde)
- Luftverschmutzung Tagesdaten von Los Angeles von 2019 (Corona Jahre vernachlässigen)

In [1]:
'Import der Module'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as requests
import json

In [21]:
#API Sign-Up
email="felixsch00@outlook.de"
#response_signup= requests.get(f"https://aqs.epa.gov/data/api/signup?email={email}")
#API Key via E-Mail
api_key="goldfox88" 
#Check if API is available --> 200 is working
response_available=requests.get("https://aqs.epa.gov/data/api/metaData/isAvailable")
print(f"Status Code Available: {response_available.status_code}")
#Definitions
response_definition=requests.get(f"https://aqs.epa.gov/data/api/metaData/fieldsByService?email={email}&key={api_key}&service=sampleData")
print(f"Status Code E-Mail and Key okay: {response_definition.status_code}")

#Check available parameter classes (groups of parameter) -> to a JSON File
resp_parameter_classes=requests.get(f"https://aqs.epa.gov/data/api/list/classes?email={email}&key={api_key}")
with open("API_Infos/Parameter_Classes.json","w") as file:
    json.dump(resp_parameter_classes.json(),file,indent=4)
#Parameters from AQI POLLUTANTS   
def get_parameter_from_class(class_code:str,email=email,api_key=api_key):
    """Get the code and definition of available AQ Measurements, stores them in a JSON-File in API_Data Folder

    Args:
        class_code (str): Class Code shown in the Parameters_Classes.json file
        email (_type_, optional): For API. Defaults to email.
        api_key (_type_, optional): For API. Defaults to api_key.
    """
    parameters_api={"email":email,"key":api_key,"pc":class_code}
    req_parameters=requests.get("https://aqs.epa.gov/data/api/list/parametersByClass?",params=parameters_api)
    with open(f"API_Infos/Parameter_{class_code}.json","w") as file:
        json.dump(req_parameters.json(),file,indent=4)
    print(f"{class_code} Parameters successfully print to json file")
    return 

#Basic Pollutants
get_parameter_from_class("AQI POLLUTANTS")
#Meteorological Parameters
get_parameter_from_class("MET")
#Volatile organic compounds
get_parameter_from_class("VOC")



Status Code Available: 200
Status Code E-Mail and Key okay: 200
AQI POLLUTANTS Parameters successfully print to json file
MET Parameters successfully print to json file
VOC Parameters successfully print to json file


## Daten 

### Welche zeitliche Intervalle soll ich in meiner Analyse beobachten ?
- es können Daten bis zu den Messintervallen abgerufen werden (SampleData) -> damit könnte ich eigenen Intervalle (z.B. 6h) durch Mittelwertbildung berechnen
- einfacher ist es einfach die **täglichen Daten** zu betrachten (für eine genauere Betrachtung können ja auch die Maxima/Minima betrachtet werden) --> Daily Summary Data by Site

### Parameter

In dem Codeblock oben wurden die allgemein verfügbaren Messsubstanzen in JSON Files geschrieben. 

### Messstadionen
- wichtig ist die Auswahl von drei zentral gelegenen Messstadionen. In einer [Kartenansicht](https://epa.maps.arcgis.com/apps/webappviewer/index.html?id=5f239fd3e72f424f98ef3d5def547eb5&extent=-146.2334,13.1913,-46.3896,56.5319) wurden diese ausgewählt.
    + Los Angeles-North Main Street (Short: LA_N): AQS Site ID 06-037-1103
    + Pico Rivera : AQS Site ID	06-037-1602
    + Compton: AQS Site ID	06-037-1302

![Die drei makierten Messstadionen wurden ausgesucht.](Stations_Map.png) 

In [6]:
request_la_n=requests.get(
    f"https://aqs.epa.gov/data/api/dailyData/bySite?email={email}&key={api_key}&param=44201&bdate=20150101&edate=20150131&cbdate=20191231&state=06&county=037&site=1103")
request_la_n.json()

{'Header': [{'status': 'Success',
   'request_time': '2022-11-24T03:49:46-05:00',
   'url': 'https://aqs.epa.gov/data/api/dailyData/bySite?email=felixsch00@outlook.de&key=goldfox88&param=44201&bdate=20150101&edate=20150131&cbdate=20191231&state=06&county=037&site=1103',
   'rows': 124}],
 'Data': [{'state_code': '06',
   'county_code': '037',
   'site_number': '1103',
   'parameter_code': '44201',
   'poc': 1,
   'latitude': 34.06659,
   'longitude': -118.22688,
   'datum': 'WGS84',
   'parameter': 'Ozone',
   'sample_duration_code': '1',
   'sample_duration': '1 HOUR',
   'pollutant_standard': 'Ozone 1-hour 1979',
   'date_local': '2015-01-01',
   'units_of_measure': 'Parts per million',
   'event_type': 'No Events',
   'observation_count': 21,
   'observation_percent': 88.0,
   'validity_indicator': 'Y',
   'arithmetic_mean': 0.021952,
   'first_max_value': 0.043,
   'first_max_hour': 13,
   'aqi': None,
   'method_code': '087',
   'method': 'INSTRUMENTAL - ULTRA VIOLET ABSORPTION',
